In [1]:
from tqdm import tqdm_notebook as tqdm
from IPython.display import Markdown, display
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow import keras

In [2]:
def printmd(string):
    display(Markdown(string))

При чтении будем хранить данные в переменной `CACHE`, чтобы ускорить загрузку. В случае их обновления, достаточно вызвать `read(override=True)`

In [3]:
SMALL_DATASET_DIR = '../lab1/notMNIST_small/'
LARGE_DATASET_DIR = '../lab1/notMNIST_large/'

LABEL_MAP = {}
INV_LABEL_MAP = {}

In [4]:
def read(data_dir):
    f_v = 0
    
    X, y = [], []
    
    for f in tqdm(os.listdir(data_dir), desc='Letter'):
        
        if not f.startswith('.'):
            img_dir = os.path.join(data_dir, f)
            
            for img in os.listdir(img_dir):
                img_path = os.path.join(img_dir, img)
                data = cv2.imread(img_path, 0)
                
                if data is None:
                    continue
                    
                X.append(data * 2 / 255 - 1)
                
                if LABEL_MAP.get(f) is None:
                    LABEL_MAP[f] = f_v
                    INV_LABEL_MAP[f_v] = f
                    f_v += 1
                    
                y.append(LABEL_MAP[f])
                
    X = np.array(X)
    y = np.array(y)
    
    return X, y

In [5]:
def get_split_data(data_dir, size=(0.7, 0.2, 0.1), random_state=23):
    X, y = read(data_dir)
    assert abs(np.sum(size) - 1.0) < 0.001
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=size[2], 
                                                        random_state=random_state, 
                                                        shuffle=True)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                      test_size=size[1], 
                                                      random_state=random_state, 
                                                      shuffle=True)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [6]:
X_train, y_train, X_val, y_val, X_test, y_test = get_split_data(SMALL_DATASET_DIR)

In [7]:
X_val.shape, y_val.shape, X_test.shape, y_test.shape

((3371, 28, 28), (3371,), (1873, 28, 28), (1873,))

In [8]:
lr = 0.1
epochs = 50
batch_size = 64

In [9]:
callbacks = [
    keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, batch_size=32, write_graph=True,
                                write_grads=True, write_images=False, embeddings_freq=0,
                                embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None,
                                update_freq='epoch'),
    keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.000001, patience=3, verbose=2, mode='auto',
                                  baseline=None,
                                  restore_best_weights=True),
    #keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=2, verbose=2,
    #                                  mode='auto', min_delta=0.001, cooldown=0, min_lr=0),
]

In [10]:
printmd('**Building with new architecture...**')

print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}\n')

model = keras.Sequential([
   keras.layers.Flatten(input_shape=(28, 28)),
   keras.layers.Dense(128, activation=tf.nn.relu),
   keras.layers.Dense(96, activation=tf.nn.relu),
   keras.layers.Dense(64, activation=tf.nn.relu),
   keras.layers.Dense(10, activation=tf.nn.softmax)
])

print(model.summary())

model.compile(optimizer=tf.keras.optimizers.SGD(lr=lr),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, y_train, 
          epochs=epochs,
          batch_size=batch_size, 
          validation_data=(X_val, y_val))
    
loss, acc = model.evaluate(X_test, y_test)

print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

Learning rate: 0.1
Epochs: 50

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 96)                12384     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                6208      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 119,722
Trainable params: 119,722
Non-trainable params: 0
_________________________________________________________________
None
Train on 13480 samples, vali

In [19]:
printmd('**Building with new architecture...**')

epochs = 50

print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28, 1)),
    keras.layers.Dense(512, activation=tf.nn.relu),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(96, activation=tf.nn.relu),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

print(model.summary())

model.compile(optimizer=tf.keras.optimizers.SGD(lr=lr),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_val = X_val.reshape(X_val.shape[0], 28, 28, 1)

model.fit(X_train, y_train, 
          epochs=epochs, 
          batch_size=batch_size,
          validation_data=(X_val, y_val))

loss, acc = model.evaluate(X_test, y_test)

print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

Learning rate: 0.0001
Epochs: 50

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_48 (Dense)             (None, 96)                24672     
_________________________________________________________________
dropout_17 (Dropout)         (None, 96)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 64)   

13480/13480 [==============================] - ETA: 3s - loss: 2.2766 - acc: 0.156 - ETA: 4s - loss: 2.2828 - acc: 0.144 - ETA: 4s - loss: 2.3035 - acc: 0.151 - ETA: 4s - loss: 2.2937 - acc: 0.150 - ETA: 4s - loss: 2.2832 - acc: 0.150 - ETA: 4s - loss: 2.2860 - acc: 0.147 - ETA: 4s - loss: 2.2827 - acc: 0.146 - ETA: 4s - loss: 2.2841 - acc: 0.145 - ETA: 4s - loss: 2.2904 - acc: 0.148 - ETA: 3s - loss: 2.2953 - acc: 0.146 - ETA: 3s - loss: 2.2946 - acc: 0.145 - ETA: 3s - loss: 2.2941 - acc: 0.146 - ETA: 3s - loss: 2.2942 - acc: 0.141 - ETA: 3s - loss: 2.2945 - acc: 0.139 - ETA: 3s - loss: 2.2934 - acc: 0.139 - ETA: 3s - loss: 2.2943 - acc: 0.136 - ETA: 2s - loss: 2.2948 - acc: 0.134 - ETA: 2s - loss: 2.2954 - acc: 0.135 - ETA: 2s - loss: 2.2939 - acc: 0.134 - ETA: 2s - loss: 2.2953 - acc: 0.133 - ETA: 2s - loss: 2.2960 - acc: 0.134 - ETA: 2s - loss: 2.2952 - acc: 0.135 - ETA: 2s - loss: 2.2934 - acc: 0.136 - ETA: 2s - loss: 2.2923 - acc: 0.135 - ETA: 2s - loss: 2.2914 - acc: 0.135 - ETA

13480/13480 [==============================] - ETA: 4s - loss: 2.2165 - acc: 0.125 - ETA: 3s - loss: 2.2225 - acc: 0.156 - ETA: 3s - loss: 2.2309 - acc: 0.152 - ETA: 3s - loss: 2.2253 - acc: 0.146 - ETA: 3s - loss: 2.2223 - acc: 0.157 - ETA: 3s - loss: 2.2230 - acc: 0.164 - ETA: 3s - loss: 2.2244 - acc: 0.169 - ETA: 3s - loss: 2.2219 - acc: 0.170 - ETA: 3s - loss: 2.2210 - acc: 0.174 - ETA: 3s - loss: 2.2199 - acc: 0.175 - ETA: 3s - loss: 2.2235 - acc: 0.177 - ETA: 3s - loss: 2.2227 - acc: 0.175 - ETA: 2s - loss: 2.2222 - acc: 0.174 - ETA: 2s - loss: 2.2216 - acc: 0.177 - ETA: 2s - loss: 2.2216 - acc: 0.177 - ETA: 2s - loss: 2.2234 - acc: 0.177 - ETA: 2s - loss: 2.2243 - acc: 0.175 - ETA: 2s - loss: 2.2240 - acc: 0.176 - ETA: 2s - loss: 2.2240 - acc: 0.176 - ETA: 2s - loss: 2.2232 - acc: 0.177 - ETA: 2s - loss: 2.2208 - acc: 0.179 - ETA: 2s - loss: 2.2227 - acc: 0.177 - ETA: 2s - loss: 2.2210 - acc: 0.178 - ETA: 2s - loss: 2.2215 - acc: 0.177 - ETA: 2s - loss: 2.2206 - acc: 0.178 - ETA

13480/13480 [==============================] - ETA: 4s - loss: 2.0997 - acc: 0.250 - ETA: 3s - loss: 2.1198 - acc: 0.281 - ETA: 3s - loss: 2.1478 - acc: 0.258 - ETA: 3s - loss: 2.1495 - acc: 0.253 - ETA: 3s - loss: 2.1575 - acc: 0.247 - ETA: 3s - loss: 2.1607 - acc: 0.245 - ETA: 3s - loss: 2.1637 - acc: 0.236 - ETA: 3s - loss: 2.1578 - acc: 0.238 - ETA: 3s - loss: 2.1566 - acc: 0.237 - ETA: 3s - loss: 2.1561 - acc: 0.238 - ETA: 3s - loss: 2.1561 - acc: 0.235 - ETA: 3s - loss: 2.1542 - acc: 0.234 - ETA: 3s - loss: 2.1564 - acc: 0.234 - ETA: 3s - loss: 2.1560 - acc: 0.231 - ETA: 3s - loss: 2.1582 - acc: 0.226 - ETA: 3s - loss: 2.1572 - acc: 0.227 - ETA: 3s - loss: 2.1574 - acc: 0.228 - ETA: 3s - loss: 2.1563 - acc: 0.228 - ETA: 3s - loss: 2.1569 - acc: 0.230 - ETA: 3s - loss: 2.1533 - acc: 0.235 - ETA: 3s - loss: 2.1516 - acc: 0.236 - ETA: 3s - loss: 2.1511 - acc: 0.238 - ETA: 3s - loss: 2.1503 - acc: 0.238 - ETA: 3s - loss: 2.1483 - acc: 0.242 - ETA: 3s - loss: 2.1478 - acc: 0.242 - ETA

13480/13480 [==============================] - ETA: 3s - loss: 2.0331 - acc: 0.343 - ETA: 3s - loss: 2.0603 - acc: 0.334 - ETA: 3s - loss: 2.0952 - acc: 0.295 - ETA: 3s - loss: 2.0923 - acc: 0.300 - ETA: 3s - loss: 2.0804 - acc: 0.306 - ETA: 3s - loss: 2.0879 - acc: 0.299 - ETA: 3s - loss: 2.0807 - acc: 0.302 - ETA: 3s - loss: 2.0802 - acc: 0.304 - ETA: 3s - loss: 2.0790 - acc: 0.303 - ETA: 3s - loss: 2.0785 - acc: 0.307 - ETA: 3s - loss: 2.0799 - acc: 0.303 - ETA: 2s - loss: 2.0774 - acc: 0.303 - ETA: 2s - loss: 2.0807 - acc: 0.302 - ETA: 2s - loss: 2.0752 - acc: 0.305 - ETA: 2s - loss: 2.0746 - acc: 0.304 - ETA: 2s - loss: 2.0771 - acc: 0.302 - ETA: 2s - loss: 2.0732 - acc: 0.305 - ETA: 2s - loss: 2.0732 - acc: 0.304 - ETA: 2s - loss: 2.0740 - acc: 0.304 - ETA: 2s - loss: 2.0746 - acc: 0.304 - ETA: 2s - loss: 2.0770 - acc: 0.301 - ETA: 2s - loss: 2.0765 - acc: 0.300 - ETA: 2s - loss: 2.0761 - acc: 0.300 - ETA: 2s - loss: 2.0732 - acc: 0.304 - ETA: 2s - loss: 2.0728 - acc: 0.305 - ETA

13480/13480 [==============================] - ETA: 4s - loss: 2.0292 - acc: 0.312 - ETA: 4s - loss: 2.0146 - acc: 0.296 - ETA: 4s - loss: 1.9961 - acc: 0.322 - ETA: 4s - loss: 1.9840 - acc: 0.349 - ETA: 3s - loss: 1.9760 - acc: 0.360 - ETA: 3s - loss: 1.9783 - acc: 0.368 - ETA: 3s - loss: 1.9825 - acc: 0.359 - ETA: 3s - loss: 1.9774 - acc: 0.367 - ETA: 3s - loss: 1.9781 - acc: 0.361 - ETA: 3s - loss: 1.9851 - acc: 0.362 - ETA: 3s - loss: 1.9843 - acc: 0.363 - ETA: 3s - loss: 1.9831 - acc: 0.363 - ETA: 3s - loss: 1.9882 - acc: 0.356 - ETA: 3s - loss: 1.9847 - acc: 0.360 - ETA: 3s - loss: 1.9835 - acc: 0.360 - ETA: 3s - loss: 1.9855 - acc: 0.358 - ETA: 2s - loss: 1.9841 - acc: 0.356 - ETA: 2s - loss: 1.9839 - acc: 0.356 - ETA: 2s - loss: 1.9834 - acc: 0.355 - ETA: 2s - loss: 1.9838 - acc: 0.355 - ETA: 2s - loss: 1.9846 - acc: 0.355 - ETA: 2s - loss: 1.9848 - acc: 0.355 - ETA: 2s - loss: 1.9835 - acc: 0.355 - ETA: 2s - loss: 1.9809 - acc: 0.357 - ETA: 2s - loss: 1.9828 - acc: 0.357 - ETA

13480/13480 [==============================] - ETA: 3s - loss: 1.9381 - acc: 0.453 - ETA: 3s - loss: 1.8900 - acc: 0.434 - ETA: 3s - loss: 1.8750 - acc: 0.425 - ETA: 3s - loss: 1.8747 - acc: 0.429 - ETA: 3s - loss: 1.8911 - acc: 0.424 - ETA: 3s - loss: 1.8941 - acc: 0.422 - ETA: 3s - loss: 1.8926 - acc: 0.422 - ETA: 3s - loss: 1.8895 - acc: 0.425 - ETA: 3s - loss: 1.8939 - acc: 0.420 - ETA: 3s - loss: 1.8918 - acc: 0.419 - ETA: 3s - loss: 1.8964 - acc: 0.414 - ETA: 2s - loss: 1.8934 - acc: 0.417 - ETA: 2s - loss: 1.8922 - acc: 0.418 - ETA: 2s - loss: 1.8923 - acc: 0.414 - ETA: 2s - loss: 1.8950 - acc: 0.415 - ETA: 2s - loss: 1.8953 - acc: 0.416 - ETA: 2s - loss: 1.8913 - acc: 0.419 - ETA: 2s - loss: 1.8909 - acc: 0.420 - ETA: 2s - loss: 1.8877 - acc: 0.423 - ETA: 2s - loss: 1.8865 - acc: 0.424 - ETA: 2s - loss: 1.8853 - acc: 0.424 - ETA: 2s - loss: 1.8872 - acc: 0.424 - ETA: 2s - loss: 1.8891 - acc: 0.423 - ETA: 2s - loss: 1.8878 - acc: 0.424 - ETA: 2s - loss: 1.8860 - acc: 0.426 - ETA

13480/13480 [==============================] - ETA: 6s - loss: 1.8796 - acc: 0.406 - ETA: 4s - loss: 1.7801 - acc: 0.464 - ETA: 4s - loss: 1.7924 - acc: 0.475 - ETA: 4s - loss: 1.7962 - acc: 0.468 - ETA: 4s - loss: 1.7980 - acc: 0.463 - ETA: 3s - loss: 1.7880 - acc: 0.466 - ETA: 3s - loss: 1.7873 - acc: 0.463 - ETA: 3s - loss: 1.7881 - acc: 0.460 - ETA: 3s - loss: 1.7983 - acc: 0.455 - ETA: 3s - loss: 1.7974 - acc: 0.458 - ETA: 3s - loss: 1.7926 - acc: 0.460 - ETA: 3s - loss: 1.7933 - acc: 0.459 - ETA: 3s - loss: 1.7933 - acc: 0.459 - ETA: 3s - loss: 1.7923 - acc: 0.462 - ETA: 3s - loss: 1.7933 - acc: 0.464 - ETA: 3s - loss: 1.7876 - acc: 0.468 - ETA: 3s - loss: 1.7840 - acc: 0.472 - ETA: 3s - loss: 1.7855 - acc: 0.470 - ETA: 3s - loss: 1.7814 - acc: 0.472 - ETA: 3s - loss: 1.7839 - acc: 0.471 - ETA: 3s - loss: 1.7835 - acc: 0.471 - ETA: 3s - loss: 1.7866 - acc: 0.471 - ETA: 2s - loss: 1.7883 - acc: 0.471 - ETA: 2s - loss: 1.7883 - acc: 0.470 - ETA: 2s - loss: 1.7896 - acc: 0.468 - ETA

13480/13480 [==============================] - ETA: 3s - loss: 1.6458 - acc: 0.500 - ETA: 3s - loss: 1.6796 - acc: 0.534 - ETA: 3s - loss: 1.6728 - acc: 0.510 - ETA: 3s - loss: 1.6627 - acc: 0.515 - ETA: 3s - loss: 1.6720 - acc: 0.511 - ETA: 3s - loss: 1.6759 - acc: 0.512 - ETA: 3s - loss: 1.6684 - acc: 0.514 - ETA: 3s - loss: 1.6732 - acc: 0.509 - ETA: 3s - loss: 1.6698 - acc: 0.510 - ETA: 2s - loss: 1.6738 - acc: 0.506 - ETA: 2s - loss: 1.6778 - acc: 0.506 - ETA: 2s - loss: 1.6829 - acc: 0.502 - ETA: 2s - loss: 1.6919 - acc: 0.498 - ETA: 2s - loss: 1.6888 - acc: 0.498 - ETA: 2s - loss: 1.6944 - acc: 0.495 - ETA: 2s - loss: 1.6961 - acc: 0.495 - ETA: 2s - loss: 1.6975 - acc: 0.495 - ETA: 2s - loss: 1.7018 - acc: 0.492 - ETA: 2s - loss: 1.7017 - acc: 0.493 - ETA: 2s - loss: 1.7023 - acc: 0.492 - ETA: 2s - loss: 1.7049 - acc: 0.490 - ETA: 2s - loss: 1.7056 - acc: 0.489 - ETA: 2s - loss: 1.7075 - acc: 0.489 - ETA: 2s - loss: 1.7059 - acc: 0.488 - ETA: 2s - loss: 1.7060 - acc: 0.488 - ETA

In [12]:
lr = 0.0001
epochs = 30
printmd('**Building with new architecture...**')

print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')



model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation=tf.nn.tanh),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
loss_method = 'sparse_categorical_crossentropy'

model.compile(optimizer=tf.train.AdamOptimizer(lr), 
              loss=loss_method,
              metrics=['accuracy'])

model.fit(X_train, y_train, 
          epochs=epochs,
          validation_data=(X_val, y_val),
          callbacks=callbacks)

loss, acc = model.evaluate(X_test, y_test)

print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

**Building with new architecture...**

Learning rate: 0.0001
Epochs: 30
Train on 13480 samples, validate on 3371 samples
Epoch 1/30
13480/13480 [==============================] - ETA: 1:40 - loss: 2.5382 - acc: 0.156 - ETA: 24s - loss: 2.4276 - acc: 0.150 - ETA: 14s - loss: 2.3931 - acc: 0.14 - ETA: 11s - loss: 2.3457 - acc: 0.14 - ETA: 10s - loss: 2.3135 - acc: 0.16 - ETA: 9s - loss: 2.2810 - acc: 0.1905 - ETA: 8s - loss: 2.2563 - acc: 0.195 - ETA: 7s - loss: 2.2122 - acc: 0.212 - ETA: 7s - loss: 2.1818 - acc: 0.221 - ETA: 7s - loss: 2.1574 - acc: 0.227 - ETA: 6s - loss: 2.1235 - acc: 0.238 - ETA: 6s - loss: 2.1026 - acc: 0.248 - ETA: 6s - loss: 2.0818 - acc: 0.256 - ETA: 6s - loss: 2.0645 - acc: 0.266 - ETA: 6s - loss: 2.0478 - acc: 0.273 - ETA: 6s - loss: 2.0359 - acc: 0.279 - ETA: 6s - loss: 2.0015 - acc: 0.297 - ETA: 6s - loss: 1.9768 - acc: 0.309 - ETA: 5s - loss: 1.9597 - acc: 0.317 - ETA: 6s - loss: 1.9500 - acc: 0.321 - ETA: 6s - loss: 1.9364 - acc: 0.327 - ETA: 5s - loss: 1.9151 - acc: 0.335 - ETA: 5s - loss: 1.90

13480/13480 [==============================] - ETA: 5s - loss: 0.3771 - acc: 0.906 - ETA: 4s - loss: 0.4127 - acc: 0.901 - ETA: 5s - loss: 0.3994 - acc: 0.896 - ETA: 4s - loss: 0.4563 - acc: 0.883 - ETA: 4s - loss: 0.4496 - acc: 0.879 - ETA: 4s - loss: 0.4592 - acc: 0.870 - ETA: 4s - loss: 0.4570 - acc: 0.869 - ETA: 4s - loss: 0.4754 - acc: 0.862 - ETA: 4s - loss: 0.4686 - acc: 0.863 - ETA: 4s - loss: 0.4628 - acc: 0.865 - ETA: 4s - loss: 0.4567 - acc: 0.866 - ETA: 3s - loss: 0.4467 - acc: 0.869 - ETA: 3s - loss: 0.4445 - acc: 0.870 - ETA: 3s - loss: 0.4471 - acc: 0.871 - ETA: 3s - loss: 0.4447 - acc: 0.873 - ETA: 3s - loss: 0.4451 - acc: 0.872 - ETA: 3s - loss: 0.4400 - acc: 0.874 - ETA: 3s - loss: 0.4380 - acc: 0.874 - ETA: 3s - loss: 0.4371 - acc: 0.876 - ETA: 3s - loss: 0.4349 - acc: 0.876 - ETA: 3s - loss: 0.4357 - acc: 0.876 - ETA: 3s - loss: 0.4416 - acc: 0.873 - ETA: 3s - loss: 0.4405 - acc: 0.874 - ETA: 3s - loss: 0.4433 - acc: 0.874 - ETA: 3s - loss: 0.4429 - acc: 0.874 - ETA

13480/13480 [==============================] - ETA: 7s - loss: 0.2864 - acc: 0.937 - ETA: 5s - loss: 0.3480 - acc: 0.881 - ETA: 5s - loss: 0.3969 - acc: 0.875 - ETA: 4s - loss: 0.4211 - acc: 0.867 - ETA: 4s - loss: 0.4091 - acc: 0.873 - ETA: 4s - loss: 0.3996 - acc: 0.876 - ETA: 4s - loss: 0.4037 - acc: 0.877 - ETA: 4s - loss: 0.4227 - acc: 0.873 - ETA: 3s - loss: 0.4097 - acc: 0.879 - ETA: 3s - loss: 0.3967 - acc: 0.881 - ETA: 3s - loss: 0.3901 - acc: 0.883 - ETA: 3s - loss: 0.3807 - acc: 0.884 - ETA: 3s - loss: 0.3873 - acc: 0.884 - ETA: 3s - loss: 0.3837 - acc: 0.883 - ETA: 3s - loss: 0.3778 - acc: 0.884 - ETA: 3s - loss: 0.3749 - acc: 0.886 - ETA: 3s - loss: 0.3692 - acc: 0.888 - ETA: 3s - loss: 0.3683 - acc: 0.888 - ETA: 3s - loss: 0.3688 - acc: 0.888 - ETA: 3s - loss: 0.3736 - acc: 0.887 - ETA: 3s - loss: 0.3717 - acc: 0.887 - ETA: 3s - loss: 0.3783 - acc: 0.886 - ETA: 3s - loss: 0.3812 - acc: 0.885 - ETA: 3s - loss: 0.3800 - acc: 0.887 - ETA: 2s - loss: 0.3827 - acc: 0.886 - ETA

13480/13480 [==============================] - ETA: 5s - loss: 0.1824 - acc: 0.968 - ETA: 5s - loss: 0.2446 - acc: 0.931 - ETA: 4s - loss: 0.2480 - acc: 0.925 - ETA: 5s - loss: 0.2474 - acc: 0.921 - ETA: 4s - loss: 0.2741 - acc: 0.909 - ETA: 4s - loss: 0.2920 - acc: 0.905 - ETA: 4s - loss: 0.2935 - acc: 0.904 - ETA: 4s - loss: 0.3020 - acc: 0.904 - ETA: 4s - loss: 0.3097 - acc: 0.900 - ETA: 4s - loss: 0.3089 - acc: 0.900 - ETA: 4s - loss: 0.3106 - acc: 0.899 - ETA: 3s - loss: 0.3100 - acc: 0.901 - ETA: 3s - loss: 0.3270 - acc: 0.897 - ETA: 3s - loss: 0.3301 - acc: 0.895 - ETA: 3s - loss: 0.3196 - acc: 0.899 - ETA: 3s - loss: 0.3143 - acc: 0.901 - ETA: 3s - loss: 0.3142 - acc: 0.901 - ETA: 3s - loss: 0.3238 - acc: 0.899 - ETA: 3s - loss: 0.3214 - acc: 0.901 - ETA: 3s - loss: 0.3265 - acc: 0.898 - ETA: 3s - loss: 0.3303 - acc: 0.898 - ETA: 3s - loss: 0.3327 - acc: 0.898 - ETA: 3s - loss: 0.3366 - acc: 0.897 - ETA: 3s - loss: 0.3323 - acc: 0.899 - ETA: 3s - loss: 0.3338 - acc: 0.898 - ETA

13480/13480 [==============================] - ETA: 8s - loss: 0.3551 - acc: 0.937 - ETA: 5s - loss: 0.2056 - acc: 0.937 - ETA: 4s - loss: 0.2566 - acc: 0.927 - ETA: 4s - loss: 0.2360 - acc: 0.934 - ETA: 4s - loss: 0.2323 - acc: 0.933 - ETA: 4s - loss: 0.2498 - acc: 0.930 - ETA: 3s - loss: 0.2462 - acc: 0.927 - ETA: 3s - loss: 0.2576 - acc: 0.923 - ETA: 3s - loss: 0.2724 - acc: 0.916 - ETA: 3s - loss: 0.2764 - acc: 0.916 - ETA: 3s - loss: 0.2762 - acc: 0.917 - ETA: 3s - loss: 0.2786 - acc: 0.917 - ETA: 3s - loss: 0.2739 - acc: 0.918 - ETA: 3s - loss: 0.2676 - acc: 0.921 - ETA: 3s - loss: 0.2737 - acc: 0.918 - ETA: 3s - loss: 0.2694 - acc: 0.920 - ETA: 3s - loss: 0.2701 - acc: 0.919 - ETA: 3s - loss: 0.2718 - acc: 0.919 - ETA: 3s - loss: 0.2744 - acc: 0.918 - ETA: 3s - loss: 0.2812 - acc: 0.915 - ETA: 3s - loss: 0.2799 - acc: 0.916 - ETA: 3s - loss: 0.2781 - acc: 0.916 - ETA: 3s - loss: 0.2800 - acc: 0.915 - ETA: 3s - loss: 0.2820 - acc: 0.915 - ETA: 3s - loss: 0.2926 - acc: 0.912 - ETA

13480/13480 [==============================] - ETA: 4s - loss: 0.1730 - acc: 0.906 - ETA: 4s - loss: 0.2176 - acc: 0.932 - ETA: 4s - loss: 0.2522 - acc: 0.917 - ETA: 4s - loss: 0.2472 - acc: 0.916 - ETA: 4s - loss: 0.2379 - acc: 0.919 - ETA: 4s - loss: 0.2438 - acc: 0.920 - ETA: 4s - loss: 0.2473 - acc: 0.919 - ETA: 4s - loss: 0.2518 - acc: 0.919 - ETA: 4s - loss: 0.2452 - acc: 0.919 - ETA: 4s - loss: 0.2522 - acc: 0.916 - ETA: 4s - loss: 0.2431 - acc: 0.919 - ETA: 4s - loss: 0.2372 - acc: 0.922 - ETA: 4s - loss: 0.2450 - acc: 0.918 - ETA: 4s - loss: 0.2450 - acc: 0.919 - ETA: 4s - loss: 0.2444 - acc: 0.919 - ETA: 3s - loss: 0.2514 - acc: 0.918 - ETA: 3s - loss: 0.2534 - acc: 0.917 - ETA: 3s - loss: 0.2575 - acc: 0.916 - ETA: 3s - loss: 0.2550 - acc: 0.917 - ETA: 3s - loss: 0.2599 - acc: 0.916 - ETA: 3s - loss: 0.2566 - acc: 0.917 - ETA: 3s - loss: 0.2556 - acc: 0.917 - ETA: 3s - loss: 0.2554 - acc: 0.917 - ETA: 3s - loss: 0.2527 - acc: 0.918 - ETA: 3s - loss: 0.2526 - acc: 0.918 - ETA

13480/13480 [==============================] - ETA: 4s - loss: 0.0537 - acc: 1.000 - ETA: 4s - loss: 0.2349 - acc: 0.921 - ETA: 4s - loss: 0.2530 - acc: 0.919 - ETA: 4s - loss: 0.2541 - acc: 0.920 - ETA: 4s - loss: 0.2648 - acc: 0.918 - ETA: 4s - loss: 0.2582 - acc: 0.919 - ETA: 4s - loss: 0.2503 - acc: 0.921 - ETA: 4s - loss: 0.2400 - acc: 0.923 - ETA: 4s - loss: 0.2392 - acc: 0.922 - ETA: 3s - loss: 0.2320 - acc: 0.925 - ETA: 3s - loss: 0.2454 - acc: 0.923 - ETA: 3s - loss: 0.2391 - acc: 0.926 - ETA: 3s - loss: 0.2501 - acc: 0.922 - ETA: 3s - loss: 0.2531 - acc: 0.921 - ETA: 3s - loss: 0.2500 - acc: 0.921 - ETA: 3s - loss: 0.2460 - acc: 0.922 - ETA: 3s - loss: 0.2478 - acc: 0.921 - ETA: 3s - loss: 0.2445 - acc: 0.920 - ETA: 3s - loss: 0.2431 - acc: 0.921 - ETA: 3s - loss: 0.2440 - acc: 0.922 - ETA: 3s - loss: 0.2475 - acc: 0.922 - ETA: 3s - loss: 0.2493 - acc: 0.921 - ETA: 3s - loss: 0.2564 - acc: 0.919 - ETA: 3s - loss: 0.2525 - acc: 0.921 - ETA: 3s - loss: 0.2557 - acc: 0.920 - ETA

13480/13480 [==============================] - ETA: 5s - loss: 0.3708 - acc: 0.843 - ETA: 4s - loss: 0.2704 - acc: 0.911 - ETA: 4s - loss: 0.2677 - acc: 0.914 - ETA: 4s - loss: 0.2544 - acc: 0.923 - ETA: 4s - loss: 0.2625 - acc: 0.915 - ETA: 4s - loss: 0.2443 - acc: 0.920 - ETA: 4s - loss: 0.2639 - acc: 0.914 - ETA: 4s - loss: 0.2717 - acc: 0.913 - ETA: 3s - loss: 0.2894 - acc: 0.911 - ETA: 3s - loss: 0.2737 - acc: 0.915 - ETA: 3s - loss: 0.2665 - acc: 0.917 - ETA: 3s - loss: 0.2649 - acc: 0.917 - ETA: 3s - loss: 0.2580 - acc: 0.920 - ETA: 3s - loss: 0.2503 - acc: 0.922 - ETA: 3s - loss: 0.2490 - acc: 0.923 - ETA: 3s - loss: 0.2464 - acc: 0.924 - ETA: 3s - loss: 0.2484 - acc: 0.923 - ETA: 3s - loss: 0.2482 - acc: 0.923 - ETA: 3s - loss: 0.2474 - acc: 0.923 - ETA: 3s - loss: 0.2457 - acc: 0.924 - ETA: 3s - loss: 0.2467 - acc: 0.923 - ETA: 3s - loss: 0.2499 - acc: 0.922 - ETA: 3s - loss: 0.2522 - acc: 0.922 - ETA: 3s - loss: 0.2509 - acc: 0.922 - ETA: 3s - loss: 0.2518 - acc: 0.921 - ETA

In [1]:
def viz_clf(train_sizes, val_scores, test_scores):
    plt.title('Validation and Hold out scores for N train samples')
    plt.xlabel('N')
    plt.ylabel('Score')
    plt.plot(train_sizes, val_scores, color='green', label='valid')
    plt.plot(train_sizes, test_scores, color='blue', label='hold out')
    plt.legend()
    plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
viz_clf(train_sizes, val_scores, test_scores)